In [ ]:
import pandas as pd
import geonamescache
import unidecode
import re

In [ ]:
f = open("data/headlines.txt")
headlines = f.readlines()
f.close()

In [ ]:
headlines = [unidecode.unidecode(h.replace("\n","")) for h in headlines]

In [ ]:
Headlines = pd.DataFrame(headlines)

In [ ]:
Headlines.columns= ["headline"]
Headlines['UniCity'] = ''

In [6]:
gc = geonamescache.GeonamesCache()

In [7]:
cities = gc.get_cities()

In [8]:
C = pd.DataFrame([(c,unidecode.unidecode(cities[c]["name"]), cities[c]["countrycode"]) for c in cities])

In [9]:
C.columns = ("id", "city", "countrycode")

In [10]:
C["UniCity"] = [unidecode.unidecode(c) for c in C.city]

In [11]:
C.head()

,id,city,countrycode,UniCity
0,3041563,Andorra la Vella,AD,Andorra la Vella
1,290594,Umm Al Quwain City,AE,Umm Al Quwain City
2,291074,Ras Al Khaimah City,AE,Ras Al Khaimah City
3,291580,Zayed City,AE,Zayed City
4,291696,Khawr Fakkan,AE,Khawr Fakkan


In [ ]:
for c in C.UniCity:
    ss=re.compile(r'\b'+c+'\\b',flags=re.IGNORECASE)
    for i,h in Headlines.iterrows():
        if re.search(ss,h.headline) is not None:
            Headlines.iloc[i]['UniCity'] = c

In [12]:
for c in C.UniCity:
    for i,h in Headlines[Headlines.headline.str.contains(r'\b'+c+'\\b')].iterrows():
        Headlines.iloc[i]['UniCity'] = c

C:\Users\pborg\AppData\Local\Continuum\anaconda3\envs\discovering-disease-outbreaks\lib\site-packages\pandas\core\strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [13]:
Headlines.head()

,headline,UniCity
0,Zika Outbreak Hits Miami,Miami
1,Could Zika Reach New York City?,New York City
2,First Case of Zika in Miami Beach,Miami Beach
3,"Mystery Virus Spreads in Recife, Brazil",Recife
4,Dallas man comes down with case of Zika,Dallas


In [14]:
H = pd.merge(Headlines,C,on='UniCity',how='left')

In [17]:
H.drop(['UniCity','id'], axis=1,inplace=True)

In [18]:
Headlines.shape

(650, 2)

In [19]:
H.shape

(1016, 3)

    Number of lines in H > number of headlines since a number of towns have the same name across countries (.eg. Richmond, Monroe...)

In [38]:
C[ (C.UniCity=='Monroe') | (C.UniCity=='Richmond') ].groupby(['city', 'countrycode']).count()

id  UniCity
city     countrycode             
Monroe   US            4        4
Richmond AU            1        1
         CA            1        1
         US            4        4
         ZA            1        1

In [20]:
H.groupby(['city']).count().reset_index().sort_values('headline', ascending=False).head()

,city,headline,countrycode
330,Monroe,16,16
413,Richmond,14,14
367,Orange,10,10
5,Albany,10,10
437,San Jose,10,10
